In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import pickle

from src.paths import MODEL_DIR, PREPROCESSED_DATA_DIR, RAW_DATA_DIR, SUBMISSION_DIR

In [3]:
with open(MODEL_DIR / 'best_model.pkl', 'rb') as file:
    model = pickle.load(file)

In [4]:
data_test = pd.read_pickle(PREPROCESSED_DATA_DIR / 'data_test.pkl')
data_test.head()

,Id,Latitude,Longitude,Week,Dayofyear,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Tmin_1,DewPoint_1,PrecipTotal_1,DewPoint_2,PrecipTotal_2,DewPoint_2_mean_l1_w1,WetBulb_2_mean_l1_w1,Tmin_1_mean_l10_w1,Tmin_2_mean_l10_w7,Tavg_2_mean_l13_w1
0,3634,41.954690,-87.800991,27,183,0.0,1.0,0.0,58,51,0.0,50,0.0,49.0,58.0,61.0,58.714286,67.0
1,3635,41.954690,-87.800991,27,183,0.0,0.0,1.0,58,51,0.0,50,0.0,49.0,58.0,61.0,58.714286,67.0
2,3636,41.954690,-87.800991,27,183,1.0,0.0,0.0,58,51,0.0,50,0.0,49.0,58.0,61.0,58.714286,67.0
3,3650,41.974089,-87.824812,27,183,0.0,1.0,0.0,58,51,0.0,50,0.0,49.0,58.0,61.0,58.714286,67.0
4,3651,41.974089,-87.824812,27,183,0.0,0.0,1.0,58,51,0.0,50,0.0,49.0,58.0,61.0,58.714286,67.0


In [5]:
predictions = model.predict_proba(data_test.drop(['Id'], axis=1))[:,1]
data_test['proba'] = predictions
data_test.head()

,Id,Latitude,Longitude,Week,Dayofyear,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Tmin_1,DewPoint_1,PrecipTotal_1,DewPoint_2,PrecipTotal_2,DewPoint_2_mean_l1_w1,WetBulb_2_mean_l1_w1,Tmin_1_mean_l10_w1,Tmin_2_mean_l10_w7,Tavg_2_mean_l13_w1,proba
0,3634,41.954690,-87.800991,27,183,0.0,1.0,0.0,58,51,0.0,50,0.0,49.0,58.0,61.0,58.714286,67.0,0.060434
1,3635,41.954690,-87.800991,27,183,0.0,0.0,1.0,58,51,0.0,50,0.0,49.0,58.0,61.0,58.714286,67.0,0.048277
2,3636,41.954690,-87.800991,27,183,1.0,0.0,0.0,58,51,0.0,50,0.0,49.0,58.0,61.0,58.714286,67.0,0.047367
3,3650,41.974089,-87.824812,27,183,0.0,1.0,0.0,58,51,0.0,50,0.0,49.0,58.0,61.0,58.714286,67.0,0.060429
4,3651,41.974089,-87.824812,27,183,0.0,0.0,1.0,58,51,0.0,50,0.0,49.0,58.0,61.0,58.714286,67.0,0.048272


In [6]:
sample = pd.read_csv(RAW_DATA_DIR / 'sampleSubmission.csv')
predictions = data_test[['Id', 'proba']]
predictions
results = pd.merge(sample, predictions, on='Id', how='outer').fillna(0)
results['WnvPresent']=results['proba']
results.drop('proba', axis=1, inplace=True)
results

,Id,WnvPresent
0,1,0.0
1,2,0.0
2,3,0.0
3,4,0.0
4,5,0.0
...,...,...
116288,116289,0.0
116289,116290,0.0
116290,116291,0.0
116291,116292,0.0


In [7]:
results.to_csv(SUBMISSION_DIR / 'test_results.csv', index=False)

In [8]:
!kaggle competitions submit -c predict-west-nile-virus -f /home/aga/repos/west_nile_virus/data/submission/test_results.csv -m test

100%|██████████████████████████████████████| 1.36M/1.36M [00:01<00:00, 1.09MB/s]
Successfully submitted to West Nile Virus Prediction

In [9]:
s = !kaggle competitions submissions -c predict-west-nile-virus

In [10]:
public_score = s[3].split()[-2]
private_score = s[3].split()[-1]
print(f"Public score: {public_score}\nPrivate score: {private_score}")

Public score: 0.73319
Private score: 0.68343
